In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn import datasets
from collections import Counter

from joblib import dump
from sklearn.externals import joblib

#from keras.models import model_from_json

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


C:\Users\a.larav\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Se importa el Dataset sin informacion de nubes
Como se ha descrito, las imagenes descargadas de los portales de imagenes satelitales presentan información en multiples bandas. Es con esta información que los algoritmos de clasificación pueden emitir un juicio para determinar a que tipo de información puede corresponder un pixel: bosque, no busque,nubes,agua, agua_bosque,ninguno.
En este escenario se plantea probar el clasificador Naive Bayes con información de imagenes que NO contengan nubes. Se plantea este escenario con el fin de determinar el rendmiento del modelo con un problema de este estilo.

In [2]:
path = './satellite_dataset-sin-nubes.csv'

satellite = pd.read_csv(path)
satellite.head()

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno
0,203.0,131.0,36.0,21.0,61.0,46.0,1,0,0,0
1,207.0,137.0,39.0,27.0,71.0,59.0,1,0,0,0
2,393.0,325.0,217.0,200.0,254.0,176.0,1,0,0,0
3,269.0,201.0,58.0,11.0,51.0,43.0,1,0,0,0
4,194.0,218.0,112.0,103.0,153.0,130.0,1,0,0,0


# Filtro de valores negativos
Se realiza una filtro de las columnas que en alguna banda contengan un valor negativo porque el clasificador multinomial Naive bayes no funciona bien con esta información.

In [3]:
satellite = satellite[satellite["blue"] > 0]
satellite = satellite[satellite["red"] > 0]
satellite = satellite[satellite["green"] > 0]
satellite = satellite[satellite["nir"] > 0]
satellite = satellite[satellite["swir1"] > 0]
satellite = satellite[satellite["swir2"] > 0]

In [4]:
#satellite['wofs'].value_counts()
#satellite['bosque'].value_counts()
#satellite['wofs_bosque'].value_counts()
#satellite['cloud'].value_counts()
#satellite['ninguno'].value_counts()

# Se reemplaza los valores en columnas de clases
Como parte de preparación de los datos para crear el modelo se deben hacer algunos ajustes a la forma en que el dataset muestra la infiromación.Como se puede observar el dataset, las columnas de las clases se encuentran categorizadas por un valor binario. Lo que se realiza a continuación es reemplazar estos valores binarios por NaN en caso de ser "Cero" y el nombre de la clase en caso de ser "Uno".

In [5]:
# Reemplazar el valor de columna Wofs por NaN(0) y Wofs(1)
satellite['wofs'] = satellite['wofs'].replace(0,np.nan)
satellite['wofs'] = satellite['wofs'].replace(1,'wofs')


In [6]:
# Reemplazar el valor de columna bosque por NaN(0) y bosque(1)
satellite['bosque'] = satellite['bosque'].replace(0,np.nan)
satellite['bosque'] = satellite['bosque'].replace(1,'bosque')

In [7]:
# Reemplazar el valor de columna wofs_bosque por NaN(0) y wofs_bosque(1)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(0,np.nan)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(1,'wofs_bosque')

In [8]:
# Reemplazar el valor de columna ninguno por NaN(0) y ninguno(1)
satellite['ninguno'] = satellite['ninguno'].replace(0,np.nan)
satellite['ninguno'] = satellite['ninguno'].replace(1,'ninguno')

# Nueva columna de clasificación
Como ultima parte para la preparación de los datos debemos tener la clasificación en una sola columna. Procedemos a crearla con el nombre "label" y alli asignaremos el valor categorizado de cada conjunto de atributos.

In [9]:
#Se crea una nueva columna para almacenar el valor de etiqueta
satellite['label']=np.nan

#Se indica en la columna 'label', la clasificacion de la columna 'wofs'. Se esribe "Wofs" en columna 'label' si en  
#la columna 'wofs' esta este valor
for label in [col for col in satellite.columns if 'wofs' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'bosque'. Se esribe "bosque" en columna 'label' si en  
#la columna 'bosque' esta este valor
for label in [col for col in satellite.columns if 'bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'wofs_bosque'. Se esribe "wofs_bosque" en columna 'label' si en  
#la columna 'wofs_bosque' esta este valor
for label in [col for col in satellite.columns if 'wofs_bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'ninguno'. Se esribe "ninguno" en columna 'label' si en  
#la columna 'ninguno' esta este valor
for label in [col for col in satellite.columns if 'ninguno' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'cloud'. Se esribe "cloud" en columna 'label' si en  
#la columna 'cloud' esta este valor
for label in [col for col in satellite.columns if 'cloud' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
satellite.head(20)

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno,label
0,203.0,131.0,36.0,21.0,61.0,46.0,wofs,NaN,NaN,NaN,wofs
1,207.0,137.0,39.0,27.0,71.0,59.0,wofs,NaN,NaN,NaN,wofs
2,393.0,325.0,217.0,200.0,254.0,176.0,wofs,NaN,NaN,NaN,wofs
3,269.0,201.0,58.0,11.0,51.0,43.0,wofs,NaN,NaN,NaN,wofs
4,194.0,218.0,112.0,103.0,153.0,130.0,wofs,NaN,NaN,NaN,wofs
5,247.0,204.0,101.0,83.0,102.0,83.0,wofs,NaN,NaN,NaN,wofs
6,201.0,507.0,287.0,3934.0,1692.0,622.0,NaN,bosque,NaN,NaN,bosque
8,857.0,837.0,763.0,822.0,757.0,498.0,wofs,NaN,NaN,NaN,wofs
9,329.0,218.0,101.0,68.0,88.0,71.0,wofs,NaN,NaN,NaN,wofs
10,46.0,163.0,71.0,3.0,30.0,25.0,wofs,NaN,NaN,NaN,wofs


In [10]:
# Se crea el conjunto de calsificacion y el de entrenamiento
Y = satellite["label"]
# X = satellite.drop(columns=["label","wofs","bosque","wofs_bosque","cloud","ninguno"], axis=1)
X = satellite.drop(columns=["label","wofs","bosque","wofs_bosque","ninguno"], axis=1)
Y.value_counts()

wofs           708921
bosque         192741
ninguno         28991
wofs_bosque       450
Name: label, dtype: int64

# Se crean los conjuntos de entrenamiento y de prueba
Para hacer esto se deben dividir los datos en 4 conjuntos de datos de la siguiente manera: Conjuntos de entrenamiento(X_train, y_train) y Conjunto de test(X_test, y_test)

In [11]:
# Se dividen los datos en los conjuntos de Train y test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.3,
    random_state=0
)

In [12]:
X_test

,blue,green,red,nir,swir1,swir2
790398,215.0,126.0,21.0,8.0,51.0,40.0
220120,78.0,238.0,114.0,1753.0,706.0,243.0
300512,373.0,306.0,184.0,153.0,158.0,122.0
408935,218.0,207.0,122.0,112.0,154.0,128.0
228058,268.0,179.0,80.0,64.0,99.0,76.0
...,...,...,...,...,...,...
156009,107.0,377.0,163.0,3289.0,1037.0,342.0
561845,311.0,220.0,107.0,78.0,110.0,88.0
505797,149.0,381.0,206.0,3086.0,1248.0,435.0
591760,153.0,382.0,215.0,2857.0,1233.0,454.0


In [13]:
y_test

790398      wofs
220120    bosque
300512      wofs
408935      wofs
228058      wofs
           ...  
156009    bosque
561845      wofs
505797    bosque
591760    bosque
637644    bosque
Name: label, Length: 279331, dtype: object

# Naive Bayes para multiples capas
La clasificación multicapa del algoritmo Naive Bayes es conocida como "Clasificación Naive Bayes Multinomial". Como se puede apreciar la clasificación de nuestro conjunto de caracteristicas puede tener 4 valores: wofs, bosque, wofs_bosque y ninguno. 
En esta parte de creación del modelo se evaluaran dos variaciones del clasificador Naive Bayes: Gausiano y multinomial. En el analisis de resultados se contrastaran los resultados obtenidos para cada uno de ellos.

# Multinomial

In [14]:
#Create a Gaussian Classifier
gnb = MultinomialNB()

In [15]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [17]:
Counter(y_pred)

Counter({'wofs': 212598, 'bosque': 58158, 'ninguno': 7785, 'wofs_bosque': 790})

In [18]:
confusion_matrix(y_test, y_pred)

array([[ 57334,    233,      0,     88],
       [   824,   6422,   1270,     88],
       [     0,   1126, 211310,    514],
       [     0,      4,     18,    100]], dtype=int64)

In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.99      0.99      0.99     57655
     ninguno       0.82      0.75      0.78      8604
        wofs       0.99      0.99      0.99    212950
 wofs_bosque       0.13      0.82      0.22       122

    accuracy                           0.99    279331
   macro avg       0.73      0.89      0.75    279331
weighted avg       0.99      0.99      0.99    279331



In [20]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9850893742549162


In [21]:
#toBePersisted = dict({
#    'metadata': {
#        'name': 'NB_Multinomial'
#    }
#})
#dump(toBePersisted, 'NB_Multinomial.joblib')

In [22]:
# joblib.dump(gnb, 'NB_Multinomial.joblib') 
#joblib.dump(gnb, 'model_nb_multinomial.joblib') 

# Gaussiano

In [23]:
#Create a Gaussian Classifier
gnb = GaussianNB()

In [24]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [26]:
Counter(y_pred)

Counter({'wofs': 206158,
         'bosque': 54375,
         'ninguno': 15622,
         'wofs_bosque': 3176})

In [27]:
confusion_matrix(y_test, y_pred)

array([[ 54063,   1858,     14,   1720],
       [   298,   7973,    146,    187],
       [    12,   5767, 205946,   1225],
       [     2,     24,     52,     44]], dtype=int64)

In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.99      0.94      0.97     57655
     ninguno       0.51      0.93      0.66      8604
        wofs       1.00      0.97      0.98    212950
 wofs_bosque       0.01      0.36      0.03       122

    accuracy                           0.96    279331
   macro avg       0.63      0.80      0.66    279331
weighted avg       0.98      0.96      0.97    279331



In [29]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9595283015490583


In [30]:
#toBePersisted = dict({
#    'metadata': {
#        'name': 'NB_Multinomial'
#    }
#})
#dump(toBePersisted, 'NB_Multinomial.joblib')

In [31]:
# joblib.dump(gnb, 'NB_Multinomial.joblib') 
#joblib.dump(gnb, 'model_nb_multinomial.joblib') 

## Analisis de resultados y conclusiones

El rendmiento obtenido con el modelo Multinomial llega a ser del 98.50% y el rendimiento con el modelo Gausiano alcanza el 95.92%. Por lo que logramos determinar que el modelo Multinomial logra funcionar mejor por 3 puntos de diferencia. Revisando la matriz de confusión logramos determinar que la principal diferencia entre los modelos es que en el multinomial se confunde menos entre elementos de la clase 2(Ninguno) y la clase 3(WOFS - Agua).

**MultinomialGNB**

Accuracy: 98.50%

| Clase\Predicción | WOFS  | Bosque | Ninguno | WOFS_Bosque |
|------------------|-------|--------|---------|-------------|
| WOFS             | 57334 | 233    | 0       | 88          |
| Bosque           | 824   | 6422   | 1270    | 88          |
| Ninguno          | 0     | 1126   | 211310  | 514         |
| WOFS_Bosque      | 0     | 4      | 18      | 100         |

**GaussianMB**

Accuracy: 95.92%

| Clase\Predicción | WOFS  | Bosque | Ninguno | WOFS_Bosque |
|------------------|-------|--------|---------|-------------|
| WOFS             | 54063 | 1858   | 14      | 1720        |
| Bosque           | 298   | 7973   | 146     | 187         |
| Ninguno          | 12    | 5767   | 205946  | 1225        |
| WOFS_Bosque      | 2     | 24     | 52      | 44          |



El modelo Multinomial en terminos generales esta ofreciendo mejor rendimiento que el modelo Gaussiano. Esta diferencia es causada principalmente por el hecho que el modelo Gaussiano se equivoca mas para datos que se encuentran clasificados como Ningun y WOFS_bosque.
Por otro lado la clasificación “ninguno” puede estar presentando un bajo recall comparado con sus demas clases porque es una clasificación donde pueden estar una gran diversidad de valores haciendo que este modelo no generalice bien para esta clase particular.


# Referencias
https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

https://datascience.stackexchange.com/questions/40345/how-to-convert-multiple-columns-into-single-columns-in-pandas

https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra